In [1]:

# Install required packages
!pip install langchain sentence-transformers faiss-cpu transformers

In [1]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Microservice_Note.pdf")
documents = loader.load()
print(f"Loaded {len(documents)} documents")


Loaded 14 documents


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
docs = text_splitter.split_documents(documents)
print(f"Split into {len(docs)} chunks")


Split into 14 chunks


In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding_model)

vectorstore.save_local("ctse_vectorstore")
print("Vector store created and saved locally.")

C:\Users\Chamaththa Shamod\AppData\Local\Temp\ipykernel_2676\3223757572.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Vector store created and saved locally.


In [4]:
vectorstore = FAISS.load_local(
    "ctse_vectorstore",
    embedding_model,
    allow_dangerous_deserialization=True
)
print("Vector store loaded.")


Vector store loaded.


In [5]:
query = "explain Benefits of Monolithic Architecture?"
results = vectorstore.similarity_search(query, k=3)
print("\nTop 3 relevant chunks:")
for idx, res in enumerate(results, 1):
    print(f"\nChunk {idx}:\n", res.page_content)



Top 3 relevant chunks:

Chunk 1:
 Monolithic Applications
• Basic Structure
• Single-Tiered Structure: Built as a single, unified unit.
• Combined Modules: Functional modules like UI, server logic, and database interactions are 
combined.
• Design and Construction
• Modular Architecture: Follows a modular structure within a single unit, aligning with object-
oriented principles.
• Programming Constructs: Defined using language-specific constructs (e.g., Java packages).
• Build Artifacts: Built as a single artifact, such as a Java JAR file.
• Characteristics
• Inter-module Dependencies: Modules are tightly coupled and interdependent.
• Unified Deployment: Deployed as a single entity.
• Scalability
• Scalability Approach: Scaling involves replicating the entire application, not individual 
components.

Chunk 2:
 • Benefits of Monolithic Architecture
• Simplified Testing: Tools like Selenium enable end-to-end testing of the entire application.
• Ease of Deployment: Deployment involves si

In [8]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

qa_pipe = pipeline("text2text-generation", model="google/flan-t5-large", max_new_tokens=600)
llm = HuggingFacePipeline(pipeline=qa_pipe)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

test_answer = qa_chain.run(query)
print("\nSample answer:\n", test_answer)

while True:
    user_input = input("\nQuestion: ")
    if user_input.lower() == 'exit':
        print("Session ended.")
        break
    prompt = f"List all relevant points in detail: {user_input}"
    response = qa_chain.run(prompt)
    print("\nBot:", response)
    print("\n")


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (839 > 512). Running this sequence through the model will result in indexing errors



Sample answer:
 Simplified Testing



Question:  what is the microservices architecture?



Bot: Breaks application into smaller, manageable chunks. - Clear boundaries with defined APIs. - Quicker development, easier understanding and maintenance. Team Autonomy and Efficiency - Independent development of services by teams. - Full lifecycle ownership of services. - Flexibility to use different programming languages (Polyglot Development). Improved Scalability and Market Responsiveness - Independent scaling based on service needs. - Hardware optimization for resource requirements. - Faster product delivery and improved time to market.





Question:  List at least 5 characteristics of Monolithic Applications


...


Bot: Each microservice has its own database tailored to its requirements





Question:  list  Benefits of microservice Architecture.



Bot: Enhanced Development and Maintenance - Breaks application into smaller, manageable chunks. - Clear boundaries with defined APIs. - Quicker development, easier understanding and maintenance. Team Autonomy and Efficiency - Independent development of services by teams. - Full lifecycle ownership of services. - Flexibility to use different programming languages (Polyglot Development). Improved Scalability and Market Responsiveness - Independent scaling based on service needs. - Hardware optimization for resource requirements. - Faster product delivery and improved time to market.





Question:  exit


Session ended.
